# Exploratory notebook

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os
import urllib.parse
import gzip
from io import StringIO
# import seaborn as sns
# import matplotlib.pyplot as plt
# import scipy.stats as stats
# from scipy.stats import chi2_contingency


from io import BytesIO

# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, precision_score, recall_score
# from sklearn.tree import DecisionTreeClassifier, plot_tree
# from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
# from IPython.display import display, display_html
# from sklearn.dummy import DummyClassifier
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import LabelEncoder
# from sklearn.ensemble import RandomForestClassifier

# ACQUIRE

In [5]:

# Replace this with the actual URL-style file path to your compressed text file
file_path = "file:///Users/miattas/Downloads/anonymized-curriculum-access.txt.gz"

# Convert URL-style file path to standard file path
parsed_path = urllib.parse.urlparse(file_path)
decoded_path = urllib.parse.unquote(parsed_path.path)
standard_path = os.path.abspath(decoded_path)

# Decompress the .gz file and read its content
with gzip.open(standard_path, "rb") as gz_file:
    data = gz_file.read().decode("utf-8")

# Preprocess lines with two consecutive spaces
data = data.replace("  ", " None ")

# Create a DataFrame from the preprocessed data
df = pd.read_csv(StringIO(data), sep=" ", header=None,
                 names=["date", "time", "lesson", "user_id", "cohort_id", "ip_address"])

# Convert 'date' and 'time' columns to a single datetime column
df['date'] = pd.to_datetime(df['date'] + ' ' + df['time'])

# Drop the 'time' column if it's no longer needed
df = df.drop(['time'], axis=1)

# SQL query to join 'logs' and 'cohorts' tables
sql = """
SELECT l.date, l.time, l.path as lesson, l.user_id, c.name,
       l.ip, c.start_date, c.end_date
FROM logs l
JOIN cohorts c ON c.id=l.cohort_id;
"""



In [10]:
df.columns

Index(['date', 'lesson', 'user_id', 'cohort_id', 'ip_address'], dtype='object')

In [8]:
df.head()

,date,lesson,user_id,cohort_id,ip_address
0,2018-01-26 09:55:03,/,1,8,97.105.19.61
1,2018-01-26 09:56:02,java-ii,1,8,97.105.19.61
2,2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61
3,2018-01-26 09:56:06,slides/object_oriented_programming,1,8,97.105.19.61
4,2018-01-26 09:56:24,javascript-i/conditionals,2,22,97.105.19.61


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900223 entries, 0 to 900222
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        900223 non-null  datetime64[ns]
 1   lesson      900223 non-null  object        
 2   user_id     900223 non-null  int64         
 3   cohort_id   899803 non-null  object        
 4   ip_address  900223 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 34.3+ MB


In [11]:
df['lesson'].value_counts()


/                                                               50313
search/search_index.json                                        19519
javascript-i                                                    18983
toc                                                             18297
java-iii                                                        13733
                                                                ...  
12-distributed-ml/6.1-prepare-part-1                                1
12-distributed-ml/6.4-prepare-part-4                                1
4-python                                                            1
4-python/overview                                                   1
appendix/professional-development/post-interview-review-form        1
Name: lesson, Length: 2314, dtype: int64